In [174]:
import pandas as pd
import yfinance as yf 
import datetime as d
from prophet import Prophet 
from datetime import date,timedelta
 
start=d.date(2019, 1, 1)
end=d.date.today()
stock_df = pd.read_csv("StockStreamTickersData.csv")

tickers = stock_df["Company Name"]  

yesterday = end - timedelta(days=2)


In [175]:
investment_range = input("enter low or medium or high: ")
duration = input("enter Short-term (ST) or Long-term (LT) or Mid-term: ")
sectors_of_interest = input("enter Tech or Healthcare or Finance or Energy: ")

enter low or medium or high:  Medium
enter Short-term (ST) or Long-term (LT) or Mid-term:  Mid-term
enter Tech or Healthcare or Finance or Energy:  Finance


In [176]:
array = ['ICICI Bank Limited','State Bank of India','Kotak Mahindra Bank Limited','Bajaj Finserv Limited','Axis Bank Limited','IDBI Bank Limited',]


In [177]:
dict_csv = pd.read_csv('StockStreamTickersData.csv', header=None, index_col=0).to_dict()[1]  
symb_list = [] 
for i in array:  
    val = dict_csv.get(i)  
    symb_list.append(val)

data = yf.download(symb_list, start=yesterday, end=end)
data=data['Close']
data.reset_index(inplace=True)
melted_data = pd.melt(data, id_vars=['Date'], value_vars=symb_list, var_name='Ticker', value_name='Price')

sorted_data = melted_data.sort_values(by='Price', ascending=True)

print(sorted_data)

[*********************100%%**********************]  6 of 6 completed

        Date         Ticker        Price
5 2024-01-25        IDBI.NS    83.500000
1 2024-01-25        SBIN.NS   612.750000
0 2024-01-25   ICICIBANK.NS  1009.950012
4 2024-01-25    AXISBANK.NS  1042.250000
3 2024-01-25  BAJAJFINSV.NS  1630.150024
2 2024-01-25   KOTAKBANK.NS  1768.400024


In [178]:
new_t=[]
for i in sorted_data['Ticker']:
    new_t.append(i)
new_t


['IDBI.NS',
 'SBIN.NS',
 'ICICIBANK.NS',
 'AXISBANK.NS',
 'BAJAJFINSV.NS',
 'KOTAKBANK.NS']

In [179]:
result_df = pd.DataFrame(columns=['Ticker', 'PredictedProfit','Buying_Price','Exit_safe_price'])

In [180]:
dict_csv = pd.read_csv('StockStreamTickersData.csv', header=None, index_col=0).to_dict()[1]  

if investment_range=='Medium' and duration=='Mid-term' and sectors_of_interest=='Finance':
    a = new_t
    for i in a:
        data = yf.download(i, start=start, end=end)
        data.reset_index(inplace=True)     
        n_years = 0.5
        period = int(n_years * 365 )
        df_train = data[['Date', 'Close']]
        df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})  # rename columns

        m = Prophet()   
        m.fit(df_train) 
        future = m.make_future_dataframe(periods=period)   
        forecast = m.predict(future) 
        forecast['ds'] = pd.to_datetime(forecast['ds']).dt.date
        end = date.today()

        filtered_df = forecast[forecast['ds'] > end]
        maxx_pred=filtered_df['yhat_upper'].max()
        a=sorted_data[sorted_data['Ticker']==i]
        predicted_profit = maxx_pred - a['Price'].iloc[0]
        
        buy_price=sorted_data[sorted_data["Ticker"]==i]['Price'].iloc[0]        
        result_df.loc[len(result_df)] = [i, predicted_profit, buy_price, maxx_pred]
          

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [173]:
result_df=result_df.sort_values(by='PredictedProfit', ascending=False)
result_df

,Ticker,PredictedProfit,Buying_Price,Exit_safe_price
3,AXISBANK.NS,262.761969,1042.250000,1305.011969
5,KOTAKBANK.NS,242.523222,1768.400024,2010.923246
2,ICICIBANK.NS,125.796220,1009.950012,1135.746233
4,BAJAJFINSV.NS,125.349027,1630.150024,1755.499051
1,SBIN.NS,60.596763,612.750000,673.346763
0,IDBI.NS,1.384338,83.500000,84.884338


In [169]:
result_df[:1]['PredictedProfit'].iloc[0]

262.76196945945117

In [ ]:
result_df[:1]['Ticker'].iloc[0]

In [184]:
result_df=result_df.sort_values(by='PredictedProfit', ascending=False)
result_df.to_csv('result_summary.csv', index=False)